# Writing tests

> How to write tests with `nbmodular`

In [ ]:
%load_ext nbmodular.core.cell2func

## Writing tests with dependencies

It is best to define data variables in separate data functions, so that they can be retrieved and reused from different test functions:

In [ ]:
%%function first_data --test --data
a = 1

In [ ]:
%%function second_data --test --data
b = 2

Then we can define our test function. This function uses the variables defined in our previous data functions:

In [ ]:
%%function first_test --test
assert a+b == 3

The way this function is implemented is by retrieving first the input variables from the previous data functions, before then using them:

In [ ]:
%print test_first_test --test

def test_first_test():
    a = test_first_data()
    b = test_second_data()
    assert a+b == 3



If we try to define the same variable in another data function, we will get an error:

In [ ]:
%%function third_data --test --data
b = 3

ValueError: detected common variables with other test data functions {'b'}:

## Running tests and loading / saving